In [1]:
import pandas as pd
import numpy as np
import sys, os
import matplotlib.pyplot as plt
from IPython.display import display
from geopy.geocoders import Nominatim
from pymongo import MongoClient

In [ ]:
place = input ("Enter Address :") 
print(place)

In [ ]:
#place = "408 Brook Pine Trl, apex"
geolocator = Nominatim(user_agent="Test")
location = geolocator.geocode(place) 
print(location)

In [ ]:
print((location.latitude, location.longitude))

In [ ]:
  # Declare all variables as strings. Spaces must be replaced with '+', i.e., change 'John Smith' to 'John+Smith'.
  # Define the lat, long of the location and the year
  #lat, lon, year = 33.2164, -97.1292, 2018
  lat, lon =  location.latitude, location.longitude
  # You must request an NSRDB api key from the link above
  api_key = '5qyFRrBVjEZIGuR0WEcihqCEcg4LV8DbErgE6rze'
  # Set the attributes to extract (e.g., dhi, ghi, etc.), separated by commas.
  attributes = 'ghi,dhi,dni,wind_speed,air_temperature,solar_zenith_angle'
  #attributes = 'ghi'
  # Choose year of data
  year = '2018'
  # Set leap year to true or false. True will return leap day data if present, false will not.
  leap_year = 'false'
  # Set time interval in minutes, i.e., '30' is half hour intervals. Valid intervals are 30 & 60.
  interval = '60'
  # Specify Coordinated Universal Time (UTC), 'true' will use UTC, 'false' will use the local time zone of the data.
  # NOTE: In order to use the NSRDB data in SAM, you must specify UTC as 'false'. SAM requires the data to be in the
  # local time zone.
  utc = 'false'
  # Your full name, use '+' instead of spaces.
  your_name = 'Anthony+N'
  # Your reason for using the NSRDB.
  reason_for_use = 'testing'
  # Your affiliation
  your_affiliation = 'ECU'
  # Your email address
  your_email = 'natalea20@students.ecu.edu'
  # Please join our mailing list so we can keep you up-to-date on new developments.
  mailing_list = 'false'

  # Declare url string
  url = 'https://developer.nrel.gov/api/solar/nsrdb_psm3_download.csv?wkt=POINT({lon}%20{lat})&names={year}&leap_day={leap}&interval={interval}&utc={utc}&full_name={name}&email={email}&affiliation={affiliation}&mailing_list={mailing_list}&reason={reason}&api_key={api}&attributes={attr}'.format(year=year, lat=lat, lon=lon, leap=leap_year, interval=interval, utc=utc, name=your_name, email=your_email, mailing_list=mailing_list, affiliation=your_affiliation, reason=reason_for_use, api=api_key, attr=attributes)
  # Return just the first 2 lines to get metadata:
  info = pd.read_csv(url, nrows=1)
  # See metadata for specified properties, e.g., timezone and elevation
  timezone, elevation = info['Local Time Zone'], info['Elevation']
  
  df = pd.read_csv(url, nrows=2)
    

In [ ]:
# View metadata
info

In [ ]:
df = pd.read_csv(url, skiprows=2)
df.head()

In [ ]:
# Set the time index in the pandas dataframe:
df = df.set_index(pd.date_range('1/1/{yr}'.format(yr=year), freq=interval+'Min', periods=525600/int(interval)))

# take a look
df.shape

In [ ]:
df.head()

In [ ]:
# Print column names
df.columns.values

# Visualize the GHI Data for the Year 2018

In [ ]:
plt.plot(df.index, df['GHI'], alpha = 0.5, linewidth=.3)
plt.show()

As you can see, there are many ups and downs - these correspond to the time of day. Naturally, GHI is far lower during the night. The beginning and end of this graph is the evening and the center is when the sun is highest.

In [ ]:
zoomedin = plt.subplot()
zoomedin.margins(x=-0.498)
zoomedin.plot(df.index, df['GHI'])
zoomedin.set_title('Subset Plot')

In [ ]:
df.to_csv("sample.csv")
# Connect to MongoDB
client =  MongoClient("mongodb://localhost:27017/?readPreference=primary&appname=MongoDB%20Compass&ssl=false")
db = client['Solar']
collection = db['Local']
df.reset_index(inplace=True)
data_dict = df.to_dict("records")
# Insert collection
collection.insert_many(data_dict)

# Now, Let's Run a Linear Regression Algorithm on the Data

In [94]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import RandomizedSearchCV, cross_val_score, KFold, cross_val_predict
from sklearn.model_selection import train_test_split
import datetime as dt

## Get 3 Years of Data (2017-2019)

In [50]:
place = input ("Enter Address :") 

Enter Address :536 Garrison Ct SW Concord NC 28025


In [51]:
geolocator = Nominatim(user_agent="Test")
location = geolocator.geocode(place)

In [80]:
years = ['2017','2018','2019']
df = []

In [81]:
lat, lon =  location.latitude, location.longitude
api_key = '5qyFRrBVjEZIGuR0WEcihqCEcg4LV8DbErgE6rze'
attributes = 'ghi'
leap_year = 'false'
interval = '60'
utc = 'false'
your_name = 'Anthony+N'
reason_for_use = 'testing'
your_affiliation = 'ECU'
your_email = 'natalea20@students.ecu.edu'
mailing_list = 'false'
for year in years:
    url = 'https://developer.nrel.gov/api/solar/nsrdb_psm3_download.csv?wkt=POINT({lon}%20{lat})&names={year}&leap_day={leap}&interval={interval}&utc={utc}&full_name={name}&email={email}&affiliation={affiliation}&mailing_list={mailing_list}&reason={reason}&api_key={api}&attributes={attr}'.format(year=year, lat=lat, lon=lon, leap=leap_year, interval=interval, utc=utc, name=your_name, email=your_email, mailing_list=mailing_list, affiliation=your_affiliation, reason=reason_for_use, api=api_key, attr=attributes)
    df.append(pd.read_csv(url, skiprows=2))

In [82]:
df[0].head()

Year  Month  Day  Hour  Minute  GHI
0  2017      1    1     0      30    0
1  2017      1    1     1      30    0
2  2017      1    1     2      30    0
3  2017      1    1     3      30    0
4  2017      1    1     4      30    0

In [83]:
df[2].loc[df[2]['Hour'] == 12]

Year  Month  Day  Hour  Minute  GHI
12    2019      1    1    12      30  142
36    2019      1    2    12      30  248
60    2019      1    3    12      30  162
84    2019      1    4    12      30   35
108   2019      1    5    12      30  564
...    ...    ...  ...   ...     ...  ...
8652  2019     12   27    12      30  314
8676  2019     12   28    12      30  480
8700  2019     12   29    12      30  172
8724  2019     12   30    12      30  556
8748  2019     12   31    12      30  564

[365 rows x 6 columns]

In [84]:
noon_values = pd.DataFrame(columns=['Year','Month','Day','Hour','Minute','GHI'])

In [85]:
noon_values.head()

Empty DataFrame
Columns: [Year, Month, Day, Hour, Minute, GHI]
Index: []

In [86]:
noon_2017 = df[0].loc[df[0]['Hour'] == 12]
noon_values = noon_values.append(noon_2017)
noon_2018 = df[1].loc[df[1]['Hour'] == 12]
noon_values = noon_values.append(noon_2018)
noon_2019 = df[2].loc[df[2]['Hour'] == 12]
noon_values = noon_values.append(noon_2019)
noon_values.tail()

Year Month Day Hour Minute  GHI
8652  2019    12  27   12     30  314
8676  2019    12  28   12     30  480
8700  2019    12  29   12     30  172
8724  2019    12  30   12     30  556
8748  2019    12  31   12     30  564

In [87]:
# Set the time index in the pandas dataframe:
#df = df.set_index(pd.date_range('1/1/{yr}'.format(yr=year), freq=interval+'Min', periods=525600/int(interval)))
noon_values = noon_values.set_index(pd.date_range(start='1/1/2017',end='12/31/2019'))
# take a look
noon_values.tail()

Year Month Day Hour Minute  GHI
2019-12-27  2019    12  27   12     30  314
2019-12-28  2019    12  28   12     30  480
2019-12-29  2019    12  29   12     30  172
2019-12-30  2019    12  30   12     30  556
2019-12-31  2019    12  31   12     30  564

# Run the Linear Regression

In [92]:
ghi = noon_values['GHI']
date_time = noon_values.index

In [113]:
X_train, X_test, y_train, y_test = train_test_split(date_time, ghi, test_size=0.33)
k = 10

In [118]:
X_train = np.array(X_train)
#X_test = np.array(X_test)
X_test = pd.DataFrame(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test)

In [119]:
y_train = y_train.reshape(-1,1)
y_test = y_test.reshape(-1,1)
X_train = X_train.reshape(-1,1)
#X_test = X_test.reshape(-1,1)


In [120]:
model = LinearRegression()
model.fit(X_train,y_train)

LinearRegression()

In [122]:
X_test.columns = ["Date"]
X_test['Date'] = pd.to_datetime(X_test['Date'])
X_test['Date']=X_test['Date'].map(dt.datetime.toordinal)

In [123]:
ghi_prediction = model.predict(X_test)

In [124]:
ghi_prediction

array([[1455.93298145],
       [1455.93298145],
       [1455.93298145],
       [1455.93298145],
       [1455.93298145],
       [1455.93298145],
       [1455.93298145],
       [1455.93298145],
       [1455.93298145],
       [1455.93298145],
       [1455.93298145],
       [1455.93298145],
       [1455.93298145],
       [1455.93298145],
       [1455.93298145],
       [1455.93298145],
       [1455.93298145],
       [1455.93298145],
       [1455.93298145],
       [1455.93298145],
       [1455.93298145],
       [1455.93298145],
       [1455.93298145],
       [1455.93298145],
       [1455.93298145],
       [1455.93298145],
       [1455.93298145],
       [1455.93298145],
       [1455.93298145],
       [1455.93298145],
       [1455.93298145],
       [1455.93298145],
       [1455.93298145],
       [1455.93298145],
       [1455.93298145],
       [1455.93298145],
       [1455.93298145],
       [1455.93298145],
       [1455.93298145],
       [1455.93298145],
       [1455.93298145],
       [1455.932